### **Develop a system to transcribe speech from audio or video recordings into text. Python, Speech Recognition libraries. Web application or desktop software for uploading audio/video and displaying transcripts.**

### **Installing Required Packages and Dependencies**
- `!pip install Flask ngrok pyngrok SpeechRecognition pydub`: This command installs the Python packages Flask (a micro web framework), ngrok and pyngrok (for tunneling localhost to the internet), SpeechRecognition (for converting speech to text), and pydub (for audio processing).
- `!apt-get install ffmpeg`: This command installs ffmpeg, a tool for handling multimedia data, which is required by pydub to handle various audio formats.

In [1]:
!pip install Flask ngrok pyngrok SpeechRecognition pydub

!apt-get install ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 29.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


### **Setting Up Ngrok for Localhost Tunneling**

- `from pyngrok import ngrok`: Imports the ngrok module which allows you to create a secure tunnel to your localhost application.
- `!ngrok authtoken YOUR_AUTHTOKEN_HERE`: Authenticates ngrok with your personal authtoken.

### **How to Get Your ngrok Authtoken**

1. **Sign Up/Log In**: Go to the [ngrok website](https://ngrok.com/) and sign up for an account or log in if you already have one.
2. **Get Your Authtoken**: Once logged in, navigate to the "Dashboard" where you will find your authtoken.
3. **Copy and Use**: Copy your authtoken and replace `YOUR_AUTHTOKEN_HERE` with your actual authtoken in the command.

By doing this, you allow ngrok to authenticate your session and create a tunnel for your Flask application.

In [2]:
from pyngrok import ngrok

!ngrok authtoken 2hSOJxFbbrvh1BU7fdL4GXx0Xzc_291QMr3CnxUTrN37LbGEt


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


### **Importing the Libraries**

- `from flask import Flask, request, render_template_string, redirect, g`: Imports Flask and various utilities from the Flask library for creating a web application, handling HTTP requests, rendering templates directly from strings, handling redirects, and using application context.
- `import speech_recognition as sr`: Imports the SpeechRecognition library, which provides functionalities to recognize speech from audio files and convert it into text.
- `import pydub import AudioSegment`: Imports the AudioSegment class from the pydub library, which is used to manipulate audio files, including conversion between different formats.
- `import moviepy.editor as mp`: Imports the moviepy library's editor module, allowing for handling and processing video files.
- `import os`: Imports the os module, which provides functions for interacting with the operating system, such as file handling.
- `from pyngrok import ngrok`: Imports the ngrok module to create secure tunnels to localhost, exposing the Flask app to the internet.
- `import sqlite3`: Imports the sqlite3 module for interacting with SQLite databases, which is used to store and manage transcription data in this application.

These libraries collectively provide the necessary functionalities for building the Flask web application, handling audio and video files, performing speech recognition, exposing the app via ngrok, and managing a SQLite database.

In [3]:
from flask import Flask, request, render_template_string, redirect, g
import speech_recognition as sr
from pydub import AudioSegment
import moviepy.editor as mp
import os
from pyngrok import ngrok
import sqlite3


### **Setting Up the Flask Application and Upload Folder**



- `app = Flask(__name__)`: Initializes the Flask application.
- `app.config['UPLOAD_FOLDER'] = 'uploads'`: Configures the folder where uploaded files will be stored.
- `if not os.path.exists(app.config['UPLOAD_FOLDER']): os.makedirs(app.config['UPLOAD_FOLDER'])`: Checks if the upload folder exists; if not, it creates the folder. This ensures that the directory for storing uploaded files is always available.

In [4]:
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'
if not os.path.exists(app.config['UPLOAD_FOLDER']):
    os.makedirs(app.config['UPLOAD_FOLDER'])

### **Initializing the Database**


- `conn = sqlite3.connect('transcripts.db')`: Connects to the SQLite database named `transcripts.db`.
- `cursor = conn.cursor()`: Creates a cursor object to execute SQL commands.
- `cursor.execute(...)`: Executes an SQL command to create a table named `transcripts` with columns `id`, `video_name`, and `transcript`, if it doesn't already exist.
- `conn.commit()`: Commits the changes to the database.
- `conn.close()`: Closes the connection to the database.

In [5]:
def init_db():
    conn = sqlite3.connect('transcripts.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS transcripts (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            video_name TEXT,
            transcript TEXT
        )
    ''')
    conn.commit()
    conn.close()

### **Database Connection Setup**



This function is responsible for establishing a connection to the SQLite database named 'transcripts.db'. If the connection object does not already exist in the global context (i.e., `g`), it creates a new connection. It then returns the connection object for database operations.

In [6]:
def get_db():
    if 'db' not in g:
        g.db = sqlite3.connect('transcripts.db')
    return g.db

### **Audio Transcription**


This function transcribes audio files into text using the SpeechRecognition library. It first loads the audio file, splits it into 5-minute segments, and then transcribes each segment. The transcript is concatenated from the transcribed segments.

In [7]:
def transcribe_audio(file_path):
    recognizer = sr.Recognizer()

    if file_path.endswith(".mp3"):
        audio = AudioSegment.from_mp3(file_path)
    elif file_path.endswith((".mp4", ".mov", ".avi", ".mkv")):
        video = mp.VideoFileClip(file_path)
        audio = video.audio

    # Split the audio into 5-minute segments
    segment_length_ms = 600000
    segments = [audio[i:i+segment_length_ms] for i in range(0, len(audio), segment_length_ms)]

    transcript = ""
    for i, segment in enumerate(segments):
        segment.export("segment{}.wav".format(i), format="wav")
        with sr.AudioFile("segment{}.wav".format(i)) as source:
            audio_text = recognizer.listen(source)

        try:
            segment_text = recognizer.recognize_google(audio_text)
            transcript += segment_text + " "
        except sr.UnknownValueError:
            transcript += " [Unknown] "
        except sr.RequestError:
            transcript += " [Error] "

        # Clean up temporary segment file
        os.remove("segment{}.wav".format(i))

    return transcript

### **Close Database Connection**


This function is a Flask decorator that closes the database connection at the end of each request. It retrieves the database connection object from the Flask global object (`g`) and closes it if it exists.

In [8]:
@app.teardown_appcontext
def close_connection(exception=None):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

### **Index Route**


This route handles both GET and POST requests. For POST requests, it checks if a file is uploaded, saves it, transcribes it, saves the transcript to the database, and removes the file. Then, it renders the HTML template with the transcript.

In [9]:
@app.route('/', methods=['GET', 'POST'])
def index():
    transcript = ""
    if request.method == 'POST':
        if 'file' not in request.files:
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            return redirect(request.url)
        if file:
            file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
            file.save(file_path)
            transcript = transcribe_audio(file_path)
            save_transcript(file.filename, transcript)  # Save transcript to database
            os.remove(file_path)
    return render_template_string("""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>ASR Transcription</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0;
            padding: 0;
            background-color:#BFA4DB; /* Bright yellow background */
        }
        .container {
            max-width: 800px;
            margin: 50px auto;
            padding: 20px;
            background-color: #fff;
            border-radius: 10px;
            box-shadow: 0 0 20px rgba(0, 0, 0, 0.1);
            transition: all 0.3s ease; /* Smooth transition for hover effect */
        }
        .container:hover {
            transform: scale(1.05); /* Increase size on hover */
        }
        h1 {
            color: #2E0049;
            text-align: center;
            margin-bottom: 20px;
        }
        form {
            margin-top: 20px;
            text-align: center;
        }
        input[type="file"] {
            display: block;
            margin: 0 auto;
            margin-bottom: 10px;
            padding: 10px;
            border: 2px solid #4B0082; /* Green border */
            border-radius: 5px;
            background-color: #fff;
            color: #4B0082;
            font-size: 16px;
            cursor: pointer;
            transition: all 0.3s ease;
        }
        input[type="file"]:hover {
           /* Darker green on hover */
            color: #4B0082;
        }
        input[type="submit"] {
            background-color: #4B0082; /* Blue background */
            color: #fff;
            border: none;
            padding: 10px 20px;
            cursor: pointer;
            font-size: 16px;
            border-radius: 5px;
            transition: all 0.3s ease;
        }
        input[type="submit"]:hover {
            background-color: #2E0049; /* Darker blue on hover */
        }
        .transcription {
            margin-top: 30px;
            padding: 20px;
            background-color: #F2E6FF; /* Light yellow background */
            border-radius: 10px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        .transcription h2 {
            margin-bottom: 20px;
            color: #2E0049;
            font-size: 24px;
            text-align: center;
        }
        .transcription p {
            color: F8F3FF;
            font-size: 18px;
            line-height: 1.6;
            text-align: justify;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Upload Audio/Video File for Transcription</h1>
        <form method="POST" enctype="multipart/form-data">
            <input type="file" name="file" accept="audio/*,video/*">
            <input type="submit" value="Upload">
        </form>
        {% if transcript %}
            <div class="transcription">
                <h2>Transcript</h2>
                <p>{{ transcript }}</p>
            </div>
        {% endif %}
    </div>
</body>
</html>
    """, transcript=transcript)

### **Saving Transcript to Database**
This function saves the transcript of a video to the database. It takes two parameters: `video_name`, which is the name of the video, and `transcript`, which is the transcript of the video. The function connects to the database, inserts the video name and transcript into the `transcripts` table, and then commits the changes. Finally, it closes the database connection.

In [10]:
def save_transcript(video_name, transcript):
    db = get_db()
    cursor = db.cursor()
    cursor.execute("INSERT INTO transcripts (video_name, transcript) VALUES (?, ?)", (video_name, transcript))
    db.commit()
    cursor.close()

### **Initializing the Database**
This function initializes the database by creating the necessary table if it does not already exist. It connects to the SQLite database named `transcripts.db`, creates a table named `transcripts` with columns for `id` (primary key), `video_name`, and `transcript`. If the table already exists, it does nothing. Finally, it commits the changes and closes the database connection.

In [11]:
# Initialize the database
init_db()

### **Running the Flask App with ngrok Tunneling**
This code block sets up a public URL for the Flask app using ngrok, which allows accessing the local server from the internet. It connects to port 5000 where the Flask app is running, creates a tunnel, and generates a public URL. Finally, it prints the ngrok tunneling information, including the mapping from the local address to the public URL, and starts the Flask app.

In [17]:
# Run the Flask app and expose it using ngrok
public_url = ngrok.connect(5000).public_url
print(f' * ngrok tunnel "http://127.0.0.1:5000" -> "{public_url}"')
app.run()


 * ngrok tunnel "http://127.0.0.1:5000" -> "https://7c28-104-154-234-118.ngrok-free.app"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/Jun/2024 06:56:13] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2024 06:56:14] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2024 06:57:11] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2024 06:57:31] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2024 06:58:52] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2024 06:59:32] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2024 07:00:05] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2024 07:00:37] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2024 07:01:40] "POST / HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/2024 07:01:50] "POST / HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [09/Jun/20

### **Retrieving and Displaying Transcripts from Database**

This code block defines functions to retrieve transcripts from the SQLite database and display them. The `get_transcripts()` function connects to the database, executes a SELECT query to fetch all transcripts, and returns the result as a list of tuples. The `display_transcripts()` function takes the list of transcripts as input and iterates over each transcript to print its ID, video name, and transcript text. Finally, when the script is run directly, it retrieves transcripts from the database and displays them.

In [18]:
import sqlite3

def get_transcripts():
    conn = sqlite3.connect('transcripts.db')
    cursor = conn.cursor()

    cursor.execute("SELECT id, video_name, transcript FROM transcripts")
    rows = cursor.fetchall()

    conn.close()
    return rows

def display_transcripts(transcripts):
    for transcript in transcripts:
        print(f"ID: {transcript[0]}")
        print(f"Video Name: {transcript[1]}")
        print(f"Transcript: {transcript[2]}")
        print("-" * 20)

if __name__ == "__main__":
    transcripts = get_transcripts()
    display_transcripts(transcripts)


ID: 2
Video Name: Audio_05.mp3
Transcript: the value of diversity diversity is our strength and reaching our communities and broadening our perspectives it posters creativity Innovation and empathy by bringing together people from different backgrounds and experiences in a diverse environment we learn from one another difference and build more inclusive Society embracing diversity means recognizing the value of every individual and promoting equity and inclusion let's Challenge and break down barriers and create species where everyone feels value and respected by valuing diversity of the way for a richer more vibrant and harmonious world thank you 
--------------------
ID: 3
Video Name: Audio_01.mp3
Transcript: trees are amazing plant that grows all around us they are tall strong and provide us with so many benefits let's explore why trees are so important trees gives us are we breathe to take in carbon dioxide a gas we don't need internet into oxygen trees provide homes for many anima

### **Exporting Transcripts to CSV File**

This code block defines a function `export_transcripts_to_csv()` that exports transcripts from the SQLite database to a CSV file named `transcripts.csv`. The function first connects to the database and executes a SELECT query to retrieve all data from the `transcripts` table. It then retrieves the column names from the table and writes both the column headers and data rows to the CSV file using the `csv` module. Finally, the function prints a success message after the export process completes. When the script is run directly, it exports transcripts to the CSV file.

In [19]:
import sqlite3
import csv

def export_transcripts_to_csv():
    conn = sqlite3.connect('transcripts.db')
    cursor = conn.cursor()

    try:
        # Execute a SELECT statement to retrieve all data from the transcripts table
        cursor.execute("SELECT * FROM transcripts")
        rows = cursor.fetchall()

        # Get the column names from the table
        column_names = [description[0] for description in cursor.description]

        # Write the data to a CSV file
        with open('transcripts.csv', 'w', newline='', encoding='utf-8') as csv_file:
            csv_writer = csv.writer(csv_file)

            # Write the column headers
            csv_writer.writerow(column_names)

            # Write the data rows
            csv_writer.writerows(rows)

        print("Data has been successfully exported to transcripts.csv")
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()

if __name__ == "__main__":
    export_transcripts_to_csv()


Data has been successfully exported to transcripts.csv


### **Emptying Transcripts Table**

This code block defines a function `empty_transcripts_table()` that empties the `transcripts` table in the SQLite database by executing a DELETE statement to remove all rows. It first connects to the database, executes the DELETE statement, commits the changes, and then prints a message indicating the successful deletion of all records. When the script is run directly, it empties the transcripts table.

In [ ]:
import sqlite3

def empty_transcripts_table():
    conn = sqlite3.connect('transcripts.db')
    cursor = conn.cursor()

    try:
        # Execute the DELETE statement to remove all rows from the table
        cursor.execute("DELETE FROM transcripts")
        conn.commit()
        print("All records have been deleted from the transcripts table.")
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()

if __name__ == "__main__":
    empty_transcripts_table()
